In [50]:
import json_lines
import pandas as pd
import networkx as nx
from datetime import datetime

## Centralities

In [81]:
net = 'covid_en'
graph = nx.read_gexf('data/network/net_{}'.format(net))

In [72]:
deg = dict(graph.degree)
deg = {k: deg[k] for k in sorted(deg, key=deg.get, reverse=True)}

deg = pd.DataFrame.from_dict(deg, orient='index', columns=['degree']).reset_index()
deg = deg.rename(columns={'index':'hashtag'})

deg.to_csv('data/network/degree_{}.csv'.format(net),index=False)

deg.head()

,hashtag,degree
0,5g,1625
1,coronavirus,350
2,iot,194
3,italia,192
4,smartphone,168


In [82]:
PR = nx.pagerank_numpy(graph)
PR = {k: PR[k] for k in sorted(PR, key=PR.get, reverse=True)}

PR = pd.DataFrame.from_dict(PR, orient='index', columns=['PRree']).reset_index()
PR = PR.rename(columns={'index':'hashtag'})

PR.to_csv('data/network/pagerank_{}.csv'.format(net),index=False)

PR.head()

,hashtag,PRree
0,coronavirus,0.161547
1,covid19,0.055579
2,china,0.015998
3,covid,0.014470
4,coronavirusoutbreak,0.012584


In [49]:
nx.write_edgelist(graph,'data/network/net_covid_edgelist_it.csv',delimiter=',',data=['weight'])

## Text dataframes

In [20]:
df_cov, df_5g = pd.DataFrame(),pd.DataFrame()


def parse_tweet(retrieved_tweet, datetime_format='%a %b %d %H:%M:%S %z %Y'):
    # Initialize parsed tweet object
    parsed_tweet = dict()
    # Get tweet id
    parsed_tweet['tweet_id'] = str(retrieved_tweet.get('id_str'))
    # Get tweet date
    parsed_tweet['tweet_date'] = datetime.strptime(
        retrieved_tweet.get('created_at'),
        datetime_format
    )
    # Initialize parsed tweet text
    tweet_text = ''
    # Case tweet is a retweet
    if 'retweeted_status' in set(retrieved_tweet.keys()):
        # Get inner tweet
        retrieved_tweet = retrieved_tweet['retweeted_status']
    # Check if current tweet is an extended tweet
    if 'extended_tweet' in set(retrieved_tweet.keys()):
        tweet_text = retrieved_tweet['extended_tweet']['full_text']
    # Case current tweet is not an extended one
    else:
        tweet_text = retrieved_tweet['text']
    # Store tweet text
    parsed_tweet['tweet_text'] = tweet_text
    # Return tweet
    return parsed_tweet

tweets = list()
# Load input file
with open('./data/20200215_only_5g_it.jsonl', 'rb') as in_file:
    # Loop through each line in input .jsonl formatted file
    for retrieved_tweet in json_lines.reader(in_file, broken=True):
        # Format retrieved tweet according to inner DataFrame
        parsed_tweet = parse_tweet(retrieved_tweet)
        # Append parsed tweet to tweets list
        tweets.append(parsed_tweet)
# Append list of retrieved tweets to inner Dataframe
df_5g = df_5g.append(tweets, ignore_index=True)

tweets = list()
# Load input file
with open('./data/20200215_only_covid_it.jsonl', 'rb') as in_file:
    # Loop through each line in input .jsonl formatted file
    for retrieved_tweet in json_lines.reader(in_file, broken=True):
        # Format retrieved tweet according to inner DataFrame
        parsed_tweet = parse_tweet(retrieved_tweet)
        # Append parsed tweet to tweets list
        tweets.append(parsed_tweet)
# Append list of retrieved tweets to inner Dataframe
df_cov = df_cov.append(tweets, ignore_index=True)

In [2]:
df_5g.shape

(4349, 3)

In [21]:
df_cov.shape

(5000, 3)